In [1]:
from __future__ import division
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from difflib import SequenceMatcher

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
#import decisiontreeclassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
#import logisticregression classifier
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
#import knn classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

#for validating your classification model
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# feature selection
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
f1=pd.read_csv('UDCA.tsv', sep="\t")
f2=pd.read_csv('ribavirin.tsv', sep="\t")

In [6]:
f1.head()

,#node1,node2,node1_string_internal_id,node2_string_internal_id,node1_external_id,node2_external_id,neighborhood_on_chromosome,gene_fusion,phylogenetic_cooccurrence,homology,coexpression,experimentally_determined_interaction,database_annotated,automated_textmining,prediction,combined_score
0,MDM2,MDM4,1861282,1854537,9606.ENSP00000417281,9606.ENSP00000356150,0.0,0,0.0,0.786,0.0,0.999,0.9,0.856,0,0.999
1,HIF1A,TP53,1852062,1846083,9606.ENSP00000338018,9606.ENSP00000269305,0.0,0,0.0,0.000,0.0,0.963,0.9,0.948,0,0.999
2,HSPA9,TP53,1847834,1846083,9606.ENSP00000297185,9606.ENSP00000269305,0.0,0,0.0,0.000,0.0,0.993,0.0,0.868,0,0.999
3,HIF1A,CREBBP,1852062,1845124,9606.ENSP00000338018,9606.ENSP00000262367,0.0,0,0.0,0.000,0.0,0.998,0.9,0.871,0,0.999
4,CHEK1,ATM,1859362,1846577,9606.ENSP00000388648,9606.ENSP00000278616,0.0,0,0.0,0.000,0.0,0.916,0.9,0.987,0,0.999


In [7]:
f2.head()

,#node1,node2,node1_string_internal_id,node2_string_internal_id,node1_external_id,node2_external_id,neighborhood_on_chromosome,gene_fusion,phylogenetic_cooccurrence,homology,coexpression,experimentally_determined_interaction,database_annotated,automated_textmining,prediction,combined_score
0,AKT1S1,RPTOR,1852495,1848942,9606.ENSP00000341698,9606.ENSP00000307272,0.0,0,0.0,0.0,0.0,0.999,0.9,0.963,0,0.999
1,UBC,HIF1A,1852861,1852062,9606.ENSP00000344818,9606.ENSP00000338018,0.0,0,0.0,0.0,0.0,0.999,0.9,0.145,0,0.999
2,PRKDC,AKT1,1849621,1846136,9606.ENSP00000313420,9606.ENSP00000270202,0.0,0,0.0,0.0,0.0,0.869,0.9,0.958,0,0.999
3,TP53,CUL9,1846083,1844100,9606.ENSP00000269305,9606.ENSP00000252050,0.0,0,0.0,0.0,0.0,0.999,0.0,0.841,0,0.999
4,CHEK2,TP53,1857546,1846083,9606.ENSP00000372023,9606.ENSP00000269305,0.0,0,0.0,0.0,0.0,0.996,0.9,0.973,0,0.999


In [11]:
f1[f1['node2']=='UDCA']

,#node1,node2,node1_string_internal_id,node2_string_internal_id,node1_external_id,node2_external_id,neighborhood_on_chromosome,gene_fusion,phylogenetic_cooccurrence,homology,coexpression,experimentally_determined_interaction,database_annotated,automated_textmining,prediction,combined_score
311,SLCO1B1,UDCA,1844479,-31401,9606.ENSP00000256958,-1.CID000031401,0.0,0,0.0,0.0,0.0,0.0,0.70,0.828,0,0.947
318,TP53,UDCA,1846083,-31401,9606.ENSP00000269305,-1.CID000031401,0.0,0,0.0,0.0,0.0,0.0,0.70,0.818,0,0.943
334,TNFRSF10B,UDCA,1846494,-31401,9606.ENSP00000276431,-1.CID000031401,0.0,0,0.0,0.0,0.0,0.0,0.70,0.800,0,0.938
357,CYP7A1,UDCA,1848316,-31401,9606.ENSP00000301645,-1.CID000031401,0.0,0,0.0,0.0,0.0,0.0,0.15,0.915,0,0.925
437,ABCB11,UDCA,1845420,-31401,9606.ENSP00000263817,-1.CID000031401,0.0,0,0.0,0.0,0.0,0.0,0.15,0.859,0,0.877
444,cholesterol,UDCA,-5997,-31401,-1.CID000005997,-1.CID000031401,0.0,0,0.0,0.0,0.0,0.0,0.00,0.866,0,0.866
471,ABCB1,UDCA,1845794,-31401,9606.ENSP00000265724,-1.CID000031401,0.0,0,0.0,0.0,0.0,0.0,0.15,0.820,0,0.842
477,AKR1C2,UDCA,1857282,-31401,9606.ENSP00000370129,-1.CID000031401,0.0,0,0.0,0.0,0.0,0.0,0.80,0.202,0,0.835
493,BLVRA,UDCA,1845765,-31401,9606.ENSP00000265523,-1.CID000031401,0.0,0,0.0,0.0,0.0,0.0,0.15,0.800,0,0.824
494,HSPA5,UDCA,1850618,-31401,9606.ENSP00000324173,-1.CID000031401,0.0,0,0.0,0.0,0.0,0.0,0.15,0.800,0,0.824


In [10]:
f2[f2['node2']=='ribavirin']

,#node1,node2,node1_string_internal_id,node2_string_internal_id,node1_external_id,node2_external_id,neighborhood_on_chromosome,gene_fusion,phylogenetic_cooccurrence,homology,coexpression,experimentally_determined_interaction,database_annotated,automated_textmining,prediction,combined_score
217,IMPDH1,ribavirin,1852894,-37542,9606.ENSP00000345096,-1.CID000037542,0.0,0,0.0,0.0,0.0,0.0,0.80,0.939,0,0.987
309,ITPA,ribavirin,1857158,-37542,9606.ENSP00000369456,-1.CID000037542,0.0,0,0.0,0.0,0.0,0.0,0.70,0.835,0,0.949
377,IL28B,ribavirin,1860754,-37542,9606.ENSP00000409000,-1.CID000037542,0.0,0,0.0,0.0,0.0,0.0,0.00,0.915,0,0.915
384,IFNA2,ribavirin,1857173,-37542,9606.ENSP00000369554,-1.CID000037542,0.0,0,0.0,0.0,0.0,0.0,0.00,0.914,0,0.914
444,IL10,ribavirin,1860958,-37542,9606.ENSP00000412237,-1.CID000037542,0.0,0,0.0,0.0,0.0,0.0,0.15,0.852,0,0.871
446,IL2,ribavirin,1843096,-37542,9606.ENSP00000226730,-1.CID000037542,0.0,0,0.0,0.0,0.0,0.0,0.15,0.847,0,0.866
487,IFNA1,ribavirin,1846520,-37542,9606.ENSP00000276927,-1.CID000037542,0.0,0,0.0,0.0,0.0,0.0,0.00,0.828,0,0.828
491,IL8,ribavirin,1848852,-37542,9606.ENSP00000306512,-1.CID000037542,0.0,0,0.0,0.0,0.0,0.0,0.00,0.824,0,0.824
499,TP53,ribavirin,1846083,-37542,9606.ENSP00000269305,-1.CID000037542,0.0,0,0.0,0.0,0.0,0.0,0.00,0.815,0,0.815
509,MTOR,ribavirin,1854147,-37542,9606.ENSP00000354558,-1.CID000037542,0.0,0,0.0,0.0,0.0,0.0,0.00,0.800,0,0.800
